# Sentence dataset

In [1]:
from sent_dataset import SentenceDataset

## Load

In [3]:
%%time
dataset = SentenceDataset(min_len=2, max_len=20)
dataset.load('data/anna.txt', size=1000000)
print(dataset.num_tokens)
print(dataset.get_data_size())

24065
9362
CPU times: user 4.9 s, sys: 20 ms, total: 4.92 s
Wall time: 4.93 s


## Batch

In [4]:
x = dataset.get_next_batch(15)
#print(x)
print()
for t in x:
    print('[%s]' % dataset.decode(t))



[<BOS> да , я простила бы . <EOS> _ _ _ _ _ _ _ _ _ _ _ _ _ _]
[<BOS> а вот ты не считал , а рябинин считал . <EOS> _ _ _ _ _ _ _ _ _ _]
[<BOS> очнувшись , левин стал соображать , сколько скошено , сколько еще можно сделать нынче . <EOS> _ _ _ _ _]
[<BOS> но были и хорошие минуты , когда она видела одни радости , одно золото . <EOS> _ _ _ _ _]
[<BOS> я не могу быть несчастлив , но и она и он не должны быть счастливы . <EOS> _ _ _ _]
[<BOS> графиня лидия ивановна была высокая полная женщина с нездорово-желтым цветом лица и прекрасными задумчивыми черными глазами . <EOS> _ _ _]
[<BOS> он испытывал почти чувство человека неодетого в обществе . <EOS> _ _ _ _ _ _ _ _ _ _ _]
[<BOS> действительно , в эту минуту алексей александрович своею спокойною , неуклюжею походкой входил в гостиную . <EOS> _ _ _ _]
[<BOS> это испортило немного общую радость . <EOS> _ _ _ _ _ _ _ _ _ _ _ _ _ _]
[<BOS> и она поняла , что последняя надежда ее была обманута . <EOS> _ _ _ _ _ _ _ _ _]
[<BOS> теперь я должна 